## TP2 &ndash; Fonctions génératrices

Indiquez vos noms ici:

- Veroone Antonin
- Barrat Nicolas

On peut travailler dans Sage avec des séries de puissances aussi aisément qu'avec des polynômes; il suffit de déclarer notre envie de le faire.

In [1]:
R.<z> = PowerSeriesRing(QQ)  # anneau des séries entières à coefficients rationnels

Dans les faits, on ne travaille jamais avec tous les termes d'une série mais plutôt avec sa troncature à un ordre donné (20 par défaut):

In [2]:
1/(1+z)

1 - z + z^2 - z^3 + z^4 - z^5 + z^6 - z^7 + z^8 - z^9 + z^10 - z^11 + z^12 - z^13 + z^14 - z^15 + z^16 - z^17 + z^18 - z^19 + O(z^20)

mais on peut changer celui-ci à la baisse si on le souhaite:

In [3]:
1/(1+z) + O(z^5)

1 - z + z^2 - z^3 + z^4 + O(z^5)

ou encore à la hausse (mais pour cela il faut augmenter la précision au niveau de l'anneau).

In [4]:
1/(1+z) + O(z^30)  # ne fait pas ce qu'on pourrait croire

1 - z + z^2 - z^3 + z^4 - z^5 + z^6 - z^7 + z^8 - z^9 + z^10 - z^11 + z^12 - z^13 + z^14 - z^15 + z^16 - z^17 + z^18 - z^19 + O(z^20)

In [5]:
R.set_default_prec(30)

In [6]:
1/(1+z)  # ok

1 - z + z^2 - z^3 + z^4 - z^5 + z^6 - z^7 + z^8 - z^9 + z^10 - z^11 + z^12 - z^13 + z^14 - z^15 + z^16 - z^17 + z^18 - z^19 + z^20 - z^21 + z^22 - z^23 + z^24 - z^25 + z^26 - z^27 + z^28 - z^29 + O(z^30)

On peut effectuer toutes les opérations arithmétiques de base sur ces séries; et on peut passer d'une liste de coefficient à une série et vice-versa de la façon suivante.

In [7]:
f = R([1,2,3,4,5]); f

1 + 2*z + 3*z^2 + 4*z^3 + 5*z^4

In [8]:
f.padded_list()

[1, 2, 3, 4, 5]

Notez que la série considérée à l'instant n'est __pas__ tout à fait la transformée en $z$ de la suite $(1,2,3,4,5,0,0,\ldots)$: pour cela il faudrait plutôt utiliser $x = 1/z$ comme variable.

Étant donné une suite $(a_n)_{n\in\mathbf{N}}$, on appelle $\displaystyle f(x) = \sum_{n=0}^\infty a_n x^n$ la __fonction génératrice__ de la suite (sans trop se soucier ici des questions de convergence). Sa transformée en $z$ est $f(z^{-1})$ qui a de bonnes propriétés théoriques; pour le calcul numérique (comme ici) il est souvent plus simple de travailler avec $f(x)$.

Voici par exemple la fonction génératrice d'une suite célèbre:

In [9]:
R.<x> = PowerSeriesRing(QQ)

F = 1/(1 - x - x^2)  # z^2/(z^2 - z - 1)

F

1 + x + 2*x^2 + 3*x^3 + 5*x^4 + 8*x^5 + 13*x^6 + 21*x^7 + 34*x^8 + 55*x^9 + 89*x^10 + 144*x^11 + 233*x^12 + 377*x^13 + 610*x^14 + 987*x^15 + 1597*x^16 + 2584*x^17 + 4181*x^18 + 6765*x^19 + O(x^20)

## A) Partages

À la maison j'ai 42 biscuits et deux garçons: le grand les mange 3 par 3 et le petit 2 par 2. De combien de façons cela peut-on répartir les 42 biscuits en tas de 2 ou de 3 ?

Si on formalise la question, il s'agit de compter le nombre de façons d'écrire $42 = 2 a + 3 b$, soit $x^{42} = (x^2)^a (x^3)^b$. Il s'agit donc du coefficient de $x^{42}$ dans l'expression

$$ (1 + x^2 + x^4 + x^6 + \cdots)(1 + x^3 + x^6 + x^9 + \cdots) = \sum_{a = 0}^\infty (x^2)^a \cdot \sum_{b = 0}^\infty (x^3)^b = \frac{1}{1 - x^2} \cdot \frac{1}{1 - x^3}. $$

In [10]:
R.set_default_prec(50)

( 1/(1-x^2) * 1/(1-x^3) ).padded_list()[42]

8

Il y a donc 8 façons de répartir les biscuits en tas de 2 ou 3; vérifions:

In [1]:
for a in range(22):
    if (42 - 2*a) % 3 == 0:
        b = (42 - 2*a) / 3
        print (a,b)

(0, 14)
(3, 12)
(6, 10)
(9, 8)
(12, 6)
(15, 4)
(18, 2)
(21, 0)


Ce qui est bien c'est que cette méthode se généralise aisément: le nombre de façons de répartir $n$ objets en $k$ tas par paquets de tailles respectives $m_1, \ldots, m_k$ est le coefficient de $x^n$ dans la fonction génératrice

$$ \prod_{i = 1}^k \frac{1}{1 - x^{m_k}}. $$

__Question 1__. En utilisant cette méthode, déterminer le nombres de façons de répartir $10$ biscuit entre $2$ enfants:

a) si on suppose qu'on les distribue tous;

b) si on se permet d'en manger quelques-uns.

Les réponses vous semblent-elles cohérentes ?

In [12]:
def factFaim(k,m,n,f):
    somme2=1
    R.set_default_prec(n+1-f)
    for a in range(0,k):
        for i in range(0,len(m)):
            somme2=somme2*(1/(1-(x^m[i])))
    return somme2.padded_list()[n-f]

factFaim(2,[1],10,0)
factFaim(2,[1],10,4)

-1

__Question 2__. De combien de façons peut-on rendre 1,47 € en monnaie ?

In [ ]:
factFaim(1,[1,2,5,10,20,50,100],147,0)

__Question 3__. De combien de façons peut-on écrire $20$ comme la somme d'entiers positifs ? Distinguer selon que l'on tienne compte de l'ordre des termes ou pas.

In [2]:
li=[]
for i in range (1,21):
    li.append(i)
    
factFaim(1,li,20,0)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

## B) Parenthésages balancés

On appelle _mot de Dyck_ toute chaîne de caractères ne comportant que des ( et des ) et représentant une expression bien balancée (toute parenthèse ouvrante finira par se refermer un peu plus loin; et on ne peut pas refermer une paire de parenthèses avant de l'avoir ouverte).

__Question 1__. Écrire une fonction récursive __dyck($n$)__ renvoyant la liste de tous les mots de Dyck de longueur $2n$ (donc comportant $n$ paires de parenthèses). Donner la liste des 42 mots de Dyck de longueur 10.

In [3]:
def dyck(n):
    liste_mot = []
    if n == 0:
        liste_mot.append("")
    else:
        for k in range(n):
            for w1 in dyck(k):
                for w2 in dyck(n - k - 1):
                    liste_mot.append('(' + w1 + ')' + w2)
    return liste_mot

print(dyck(3))

['()()()', '()(())', '(())()', '(()())', '((()))']


__Question 2__. La longueur $C_n$ de la liste renvoyée par __dyck($n$)__ satisfait l'équation de récurrence

$$ C_{n+1} = \sum_{k=0}^n C_k C_{n-k}. $$

Utiliser celle-ci pour obtenir les valeurs de $C_n$ pour $n \in [\![ 0, 100 ] \! ]$ afin d'observer graphiquement l'allure des premiers termes de la suite.

In [4]:
def len_nb_mots_dyck(nbmax):
    liste_nb_mots = []
    for n in range(0, nbmax+1):
        len_reccur(n, liste_nb_mots)
    return liste_nb_mots

def len_reccur(n, liste_nb_mots):
    if n <= 1:
        S = 1
    else:
        S = 0
        for k in range(n):
            S += liste_nb_mots[k] * liste_nb_mots[n-k-1]
    liste_nb_mots.append(S)
    return S

# Affiche les 101 premières valeurs
#print(len_nb_mots_dyck(100))

# Affiche le graphe permettant de voir la vitesse de croissance du nombre de possibilités
#list_plot(len_nb_mots_dyck(100), plotjoined=true)

__Question 3__. Confirmer vos valeurs trouvées à la question précédente avec les 101 premiers coefficients de la fonction génératrice

$$ C(x) = \sum_{n=0}^\infty C_n x^n = \frac{1 - \sqrt{1 - 4x}}{2x}. $$

In [18]:
R.<x> = PowerSeriesRing(QQ)
R.set_default_prec(102)
# Affiche les 101 premières valeurs si souhaité
# print(((1 - sqrt(1 - 4 * x)) / (2 * x)).padded_list())

# Confirme si les valeurs trouvés à la question précédente sont les bonnes 
print("Est ce que les 101 premières valeurs trouvés à la question précédente sont les mêmes que celles de la fonction génératrice ? ")
print("Oui" if (len_nb_mots_dyck(100) == ((1 - sqrt(1 - 4 * x)) / (2 * x)).padded_list()) else "Non")

Est ce que les 101 premières valeurs trouvés à la question précédente sont les mêmes que celles de la fonction génératrice ? 
Oui
